In [1]:
import h5py
import numpy as np
import joblib
import torch

In [2]:
print(torch.cuda.is_available())

True


In [3]:
meta_data = joblib.load('/home/milo/Documents/phd/VideoMimic/PDP/data/phc_0.05/phc_act_amass_train_upright_metadata.pkl')
failed_keys = joblib.load('/home/milo/Documents/phd/VideoMimic/PDP/data/phc_0.05/failed.pkl')

In [4]:
print(meta_data.keys())

motion_lengths = np.concatenate( [ml for ml in meta_data['motion_lengths']])
keys_names = np.concatenate( [kn for kn in meta_data['key_names']])

print(len(motion_lengths))
print(sum(motion_lengths))
print(len(keys_names))
print(keys_names)
num_motions = len(keys_names)
motion_starts = np.cumsum(motion_lengths)
motion_starts = np.insert(motion_starts, 0, 0)
motion_starts = motion_starts[:-1]
print(motion_starts[:5])

dict_keys(['key_names', 'motion_lengths', 'running_mean', 'config', 'exclude_ids'])
117760
33576020
117760
['0-KIT_1747_displace_tilt_side_08_poses'
 '0-KIT_1747_displace_tilt_side_13_poses' '0-CMU_120_120_11_poses' ...
 '0-CMU_18_19_Justin_18_01_poses' '0-CMU_18_19_Justin_18_02_poses'
 '0-BioMotionLab_NTroje_rub037_0015_sitting1_poses']
[   0  309  618  927 1236]


In [5]:
print(len(failed_keys))
print(failed_keys)

222
['0-TotalCapture_s5_freestyle3_poses' '0-CMU_56_56_06_poses'
 '0-BioMotionLab_NTroje_rub011_0030_scamper_poses'
 '0-ACCAD_Male2MartialArtsExtended_c3d_Extended 3_poses'
 '0-MPI_Limits_03099_lar1_poses'
 '0-BioMotionLab_NTroje_rub010_0029_scamper_poses'
 '0-BioMotionLab_NTroje_rub104_0030_scamper_poses' '0-CMU_85_85_12_poses'
 '0-BioMotionLab_NTroje_rub078_0029_scamper_poses'
 '0-BioMotionLab_NTroje_rub017_0030_scamper_poses'
 '0-CMU_111_111_39_poses' '0-CMU_88_88_11_poses'
 '0-TotalCapture_s4_freestyle1_poses'
 '0-BioMotionLab_NTroje_rub009_0030_scamper1_poses'
 '0-BioMotionLab_NTroje_rub057_0029_scamper_poses'
 '0-Eyes_Japan_Dataset_ichige_jump-07-rope double-ichige_poses'
 '0-Eyes_Japan_Dataset_ichige_sitdown_standup-11-one knee drawn up-ichige_poses'
 '0-Eyes_Japan_Dataset_kaiwa_gesture_etc-24-matrix-kaiwa_poses'
 '0-Eyes_Japan_Dataset_kaiwa_jump-04-horizontal-kaiwa_poses'
 '0-Eyes_Japan_Dataset_kaiwa_jump-03-vertical-kaiwa_poses'
 '0-Eyes_Japan_Dataset_shiono_accident-04-damage

In [6]:
dataset_path = '/home/milo/Documents/phd/VideoMimic/PDP/data/phc_0.05/phc_act_amass_train_upright.h5'
with h5py.File(dataset_path, 'r') as hdf5_file:
    # Get the total size of the dataset
    for key in hdf5_file.keys():
        print(f"- {key}")
    dataset_size = len(hdf5_file['clean_action']) 

    assert dataset_size == sum(motion_lengths)
    # Load the `reset` boolean array
    reset = hdf5_file['reset'][:]
    action  =  hdf5_file['clean_action'][:]
    obs  =  hdf5_file['pdp_obs'][:]



- clean_action
- pdp_obs
- reset


In [7]:

exclude_ids = []
exclude_indicies = []

obs_dim = obs.shape[-1]
action_dim = action.shape[-1]

count_obs = 0
mean_obs = np.zeros(obs_dim, dtype=obs.dtype)
M2_obs = np.zeros(obs_dim, dtype=obs.dtype)
# Initialize min and max as before
obs_min = np.full(obs_dim, np.inf, dtype=obs.dtype)
obs_max = np.full(obs_dim, -np.inf, dtype=obs.dtype)


count_action = 0
mean_action = np.zeros(action_dim, dtype=obs.dtype)
M2_action = np.zeros(action_dim, dtype=obs.dtype)
# Initialize min and max as before
action_min = np.full(action_dim, np.inf, dtype=obs.dtype)
action_max = np.full(action_dim, -np.inf, dtype=obs.dtype)

for m_i in range(num_motions):

    ml = motion_lengths[m_i]
    start_idx, end_idx = motion_starts[m_i], motion_starts[m_i]+ml


    if keys_names[m_i] in failed_keys:
        exclude_ids.append(m_i)
        exclude_indicies.extend(range(start_idx, end_idx))
        continue
    

    m_r = reset[start_idx:end_idx]
    m_a = action[start_idx:end_idx]
    m_o = obs[start_idx:end_idx]

    has_no_reset = (np.sum(m_r) == 0)
    if has_no_reset:
        exclude_ids.append(m_i)
        exclude_indicies.extend(range(start_idx, end_idx))
        continue

    is_last_a_reset = (m_r[-1] == 1)
    assert is_last_a_reset, 'Last index is not Reset'

    has_only_one_reset = (np.sum(m_r) == 1)
    assert has_only_one_reset, 'More the one Reset in data'

    #OBS -------------- Update min/max
    obs_min = np.minimum(obs_min, m_o.min(axis=0))
    obs_max = np.maximum(obs_max, m_o.max(axis=0))

    # Update mean and M2 using Welford's algorithm -------------
    chunk_count = m_o.shape[0]
    
    delta = m_o - mean_obs
    new_mean_obs = mean_obs + np.sum(delta, axis=0) / (count_obs + chunk_count)
    
    delta2 = m_o - new_mean_obs
    M2_obs += np.sum(delta * delta2, axis=0)
    
    mean_obs = new_mean_obs
    count_obs += chunk_count


    #OBS -------------- Update min/max ----------------
    action_min = np.minimum(action_min, m_a.min(axis=0))
    action_max = np.maximum(action_max, m_a.max(axis=0))

    # Update mean and M2 using Welford's algorithm
    chunk_count = m_o.shape[0]
    
    delta = m_a - mean_action
    new_mean_action = mean_action + np.sum(delta, axis=0) / (count_action + chunk_count)
    
    delta2 = m_a - new_mean_action
    M2_action += np.sum(delta * delta2, axis=0)
    
    mean_action = new_mean_action
    count_action += chunk_count



all_indices = np.arange(len(action))
indices_to_keep = np.setdiff1d(all_indices, exclude_indicies)

std_obs = np.sqrt(M2_obs / count_obs)
std_action = np.sqrt(M2_action / count_action)


In [8]:
import sys
import os


module_path = os.path.abspath('/home/milo/Documents/phd/VideoMimic/PDP')

# Insert the path at the beginning of the list
sys.path.insert(0, module_path)
from pdp.utils.normalizer import LinearNormalizer

In [9]:
data = {
    'obs': {
        'min':obs_min,
        'max':obs_max,
        'mean':mean_obs,
        'std':std_obs,
    },
    'action': {
        'min':action_min,
        'max':action_max,
        'mean':mean_action,
        'std':std_action,
    },
}

normalizer = LinearNormalizer()
normalizer.fit_implicit(data=data, mode='limits')


{'min': array([-1.14937231e-01, -1.14823595e-01, -1.15022182e-01, -4.87190425e-01,
       -4.85245585e-01, -4.91551697e-01, -8.74125600e-01, -8.82958651e-01,
       -8.90396535e-01, -9.57389116e-01, -9.48450565e-01, -1.01177764e+00,
       -1.12711653e-01, -1.13021582e-01, -1.13099873e-01, -4.95233238e-01,
       -4.95254666e-01, -4.97101277e-01, -8.72371197e-01, -8.88266265e-01,
       -8.94476533e-01, -9.71106529e-01, -1.00620675e+00, -1.01220572e+00,
       -1.11675180e-01, -1.09033316e-01, -1.11933373e-01, -2.45436832e-01,
       -2.44337648e-01, -2.44577572e-01, -3.00043672e-01, -3.02813858e-01,
       -3.03009033e-01, -5.10766149e-01, -5.09986877e-01, -4.89653945e-01,
       -5.65026104e-01, -5.82555115e-01, -5.69663048e-01, -4.25661176e-01,
       -4.24264371e-01, -4.12571162e-01, -4.71986234e-01, -4.84414309e-01,
       -4.99258190e-01, -6.32363617e-01, -7.17876256e-01, -7.55014837e-01,
       -7.55731642e-01, -9.60328162e-01, -9.83743608e-01, -7.87038803e-01,
       -1.0406200

In [10]:
normalizer_state = normalizer.state_dict()

# Save the state dictionary to a file
torch.save(normalizer_state, '/home/milo/Documents/phd/VideoMimic/PDP/data/phc_0.05/normalizer_params.pt')

In [11]:
meta_data['exclude_ids'] = exclude_ids
joblib.dump(meta_data, '/home/milo/Documents/phd/VideoMimic/PDP/data/phc_0.05/phc_act_amass_train_upright_metadata.pkl')

['/home/milo/Documents/phd/VideoMimic/PDP/data/phc_0.05/phc_act_amass_train_upright_metadata.pkl']